# Data Science Project
#### Christopher Healy D00270638
#### Using Data Science to predict the popularity of Reddit posts.

# Step 1: Business Understanding
Reddit is a highly popular social media platform that is used by a wide range of individuals, entrepeneurs, and businesses. It's popularity offers the potential of a global reach to posters across an extremely diverse range of topics.

### Audience:
Many people use Reddit as a means to promote themselves or their product. This includes but is not limited to:
- Entrepeneurs advertising products to niche markets.
- Musicians/performers trying to build a platform.
- "Influencers" trying to gain recognition.

The success/popularity of a Reddit post can be measured with "Karma", a numerical value which corresponds to the total sum of ratings on a post, i.e. upvotes minus downvotes.


### Aims of the project.
Determining how a Reddit post's Karma is influenced can allow these groups to achieve the maximum reach possible. By analysing the information associated with Reddit posts and comparing it to their popularity, I hope to answer:
1. How is post length connected to post success?
2. Is date/time of posting relevant to a post's success.
3. How does the "mood" found in a post's body influence its success?
4. How do attached photos impact post performance?
5. Are there any other factors that affect how a Reddit post will be received?

Posts can be analysed to determine their "mood" from their content, this will need to be done before any machine learning models can be trained. Similarly, if post photos are included in the data they will need to be classified in some way so that their influence on post Karma can be quantitatively seen.


# Step 2: Data Mining
The data is gathered from the Reddit API using the Python Reddit API Wrapper (praw).
Post information is gathered from a variety of popular subreddits to ensure a variety of themes and audiences.
The data fetched includes a variety of information on posts submitted to the following subreddits:
- r/funny
- r/todayilearned
- r/technology
- r/aww
- r/worldnews
- r/food
- r/gaming

See separate Jupyter Notebook "DataMining.ipynb"

# Step 3 - Data Cleaning
### Datapoints used:
| Variable      | Type                | Notes                                                         |
|---------------|---------------------|---------------------------------------------------------------|
| id            | Nominal Categorical | Ignore                                                        |
| title         | Nominal Categorical | Length of title is important, otherwise ignored               |
| subreddit     | Nominal Categorical | Explanatory                                                   |
| created_utc   | Discrete Numerical  | Used to determine is_weekend, then ignored                    |
| ups           | Discrete Numerical  | Explanatory                                                   |
| downs         | Discrete Numerical  | Explanatory                                                   |
| num_comments  | Discrete Numerical  | Explanatory                                                   |
| is_video      | Nominal Categorical | Explanatory                                                   |
| selftext      | Nominal Categorical | Length of text is important, otherwise ignored                |
| is_self       | Nominal Categorical | Explanatory                                                   |
| upvote_ratio  | Continous Numerical | Explanatory                                                   |
| title_length  | Discrete Numerical  | (Calculated Manually) Explanatory                             |
| post_length   | Discrete Numerical  | (Calculated Manually) Explanatory                             |
| is_weekend_post| Nominal Categorical | Explanatory                                                   |
| score         | Discrete Numerical  | Response Variable                                             |



In [27]:
#imports
import pandas as pd
import numpy as np

In [ ]:
#load and visualise data
reddit_data = pd.read_csv("reddit_data.csv", quotechar='"', escapechar='\\')
print("Data Info:")
print(reddit_data.info())
print("Data Head:")
print(reddit_data.head())



#check for issues in the numerical columns
print(reddit_data.describe())


#check no invalid values for categorical columns
print("\nUnique subreddits:")
print(reddit_data.subreddit.unique())
print("is_video field is only true/false?:")
print(reddit_data.is_video.unique())

print("is_self field is only true/false?:")
print(reddit_data.is_self.unique())

print("is is_weekend_post being calculated properly?:")
print(reddit_data.is_weekend_post.unique())

#ensure selftext is set to an empty string when blank
reddit_data["selftext"] = reddit_data["selftext"].fillna("")


#checking for null values in other fields
print("\nChecking for null values in df:")
fields_to_check_null = ["id", "title", "subreddit", "created_utc", "ups", "downs", "num_comments", "is_video", "is_self", "score", "upvote_ratio", "title_length", "post_length", "is_weekend_post"]
for field in fields_to_check_null:
    print(f"Null {field}:" + str(reddit_data[field].isnull().sum()))


#remove unnecessary fields
reddit_data.drop("id", axis=1, inplace=True)
reddit_data.drop("title", axis=1, inplace=True)
reddit_data.drop("created_utc", axis=1, inplace=True)
reddit_data.drop("selftext", axis=1, inplace=True)


Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6867 entries, 0 to 6866
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6867 non-null   object 
 1   title            6867 non-null   object 
 2   subreddit        6867 non-null   object 
 3   created_utc      6867 non-null   float64
 4   ups              6867 non-null   int64  
 5   downs            6867 non-null   int64  
 6   num_comments     6867 non-null   int64  
 7   is_video         6867 non-null   bool   
 8   selftext         1227 non-null   object 
 9   is_self          6867 non-null   bool   
 10  score            6867 non-null   int64  
 11  upvote_ratio     6867 non-null   float64
 12  title_length     6867 non-null   int64  
 13  post_length      6867 non-null   int64  
 14  is_weekend_post  6867 non-null   bool   
dtypes: bool(3), float64(2), int64(6), object(4)
memory usage: 664.0+ KB
None
Data Head:
        id 

### Findings during data cleaning
- The info on numerical columns shown with `reddit_data.info()` show figures in reasonable ranges with no problematic max or min values.
- The categorical columns show normal value, excluding "selftext" which is unique to each post.
- When the pandas dataframe is created, blank values in the "selftext" field appeared as NaN, these needed to be changed to empty string literals.
- Investigation showed that the data had no missing / null values other than those in the selftext field, which represents a post with no textual content.

The columns of id, title, created_utc, and selftext were removed as they will not play a role in the machine learning model development.